- Für die Realisierung der Facettensuche von Medien muss das aktuelle Medium-Mapping von HTWplus angepasst werden.
- Dazu benötigt es neue Felder, mit den Werten, die hinterher aggregiert werden sollen.

In [17]:
# Delete existing Index
curl -XDELETE 'localhost:9200/htwplus?pretty'

{
  "acknowledged" : true
}


In [18]:
# 1. Eine Gruppierung nach Eigentümern (Vor- und Nachname)
# => Medium
# name => firstname, lastname
# The string field is unsupported for indexes created in 5.x in favor of the text and keyword fields.
# change all 'string' => 'text'

# Since 2.4 the Edge-N-Gram has changed so we create a new custom analyzer for Version 5
# This needs to happen at the same time we create the mapping for our type(s)
# Otherwise ES does not know about the analyzer ( analyzer not found exception )

# After that we Setup The Mapping for Our Users which now include 'firstname' + 'lastname'
# Instead of 'name'
curl -XPUT 'localhost:9200/htwplus?pretty' -H 'Content-Type: application/json' -d'
{
  "settings": {
    "analysis": {
      "analyzer": {
        "edgengram_analyzer": {
          "tokenizer": "htw_edge_ngram"
        }
      },
      "tokenizer": {
        "htw_edge_ngram": {
          "type": "edge_ngram",
          "min_gram": 2,
          "max_gram": 8,
          "token_chars": [
            "letter",
            "digit"
          ]
        }
      }
    }
  },
  "mappings": {
    "medium": {
      "properties": {
        "owner": {
          "type": "long"
        },
        "viewable": {
          "type": "long"
        },
        "public": {
          "type": "boolean"
        },
        "filename": {
          "type": "text",
          "analyzer": "edgengram_analyzer",
          "search_analyzer": "whitespace"
        }
      }
    },
    "users": {
      "properties": {
        "studycourse": {
          "index": "not_analyzed",
          "type": "text"
        },
        "role": {
          "index": "not_analyzed",
          "type": "text"
        },
        "public": {
          "type": "boolean"
        },
        "initial": {
          "type": "keyword"
        },
        "degree": {
          "index": "not_analyzed",
          "type": "text"
        },
        "firstname": {
          "search_analyzer": "whitespace",
          "analyzer": "edgengram_analyzer",
          "type": "text"
        },
        "lastname": {
          "search_analyzer": "whitespace",
          "analyzer": "edgengram_analyzer",
          "type": "text"
        },
        "semester": {
          "index": "not_analyzed",
          "type": "text"
        },
        "avatar": {
          "type": "text"
        },
        "friends": {
          "type": "long"
        }
      }
    }
  }
}'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


In [21]:
# Test The analyzer
curl -XPOST 'localhost:9200/htwplus/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "analyzer": "edgengram_analyzer",
  "text": ["Iven","Zimmer"]
}'

{
  "tokens" : [
    {
      "token" : "Iv",
      "start_offset" : 0,
      "end_offset" : 2,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "Ive",
      "start_offset" : 0,
      "end_offset" : 3,
      "type" : "word",
      "position" : 1
    },
    {
      "token" : "Iven",
      "start_offset" : 0,
      "end_offset" : 4,
      "type" : "word",
      "position" : 2
    },
    {
      "token" : "Zi",
      "start_offset" : 5,
      "end_offset" : 7,
      "type" : "word",
      "position" : 103
    },
    {
      "token" : "Zim",
      "start_offset" : 5,
      "end_offset" : 8,
      "type" : "word",
      "position" : 104
    },
    {
      "token" : "Zimm",
      "start_offset" : 5,
      "end_offset" : 9,
      "type" : "word",
      "position" : 105
    },
    {
      "token" : "Zimme",
      "start_offset" : 5,
      "end_offset" : 10,
      "type" : "word",
      "position" : 106
    },
    {
      "token" : "Zimmer",
      "start_offset" : 5,
    

In [23]:
curl -XPOST 'localhost:9200/htwplus/users?pretty' -H 'Content-Type: application/json' -d'
{
    "studycourse" : "Spezielle Anwendung - ElasticSearch",
    "role": "Administrator",
    "public": true,
    "intitial": "IZ",
    "degree": "Master",
    "firstname": "Iven",
    "Lastname": "Zimmer",
    "Semester": "0",
    "avatar": "owl.png",
    "friends": "0"
}'

{
  "_index" : "htwplus",
  "_type" : "users",
  "_id" : "AV1Aa4Yn2n88X6tmg0QN",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : true
}


In [26]:
# Verify
curl -XGET 'localhost:9200/htwplus/users/_search?pretty'

{
  "took" : 49,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "htwplus",
        "_type" : "users",
        "_id" : "AV1Aa4Yn2n88X6tmg0QN",
        "_score" : 1.0,
        "_source" : {
          "studycourse" : "Spezielle Anwendung - ElasticSearch",
          "role" : "Administrator",
          "public" : true,
          "intitial" : "IZ",
          "degree" : "Master",
          "firstname" : "Iven",
          "Lastname" : "Zimmer",
          "Semester" : "0",
          "avatar" : "owl.png",
          "friends" : "0"
        }
      }
    ]
  }
}


In [ ]:
# 2. Eine Gruppierung nach Speicherort (Root-Folder). (2 Punkte)

# 3. Eine Gruppierung nach Upload-Datum. Flexibel nach Tag, Monat oder Jahr. (4 Punkte)

# 4. Eine Gruppierung nach Dateiendungen.